In [ ]:
import pandas as pd
from datetime import datetime, timedelta

from facebook_scraper import get_posts

import smtplib, ssl
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText

import pywhatkit
import time

In [ ]:
url = []
text = []
date = []
comments = []
likes = []
user = []

interests = ['סנטר', 'צפון ישן', 'הצפון הישן', 'אחד וחצי חדרים', '1 וחצי חדרים' ,'שני חדרים' ,'בזל',
             'יחיד', 'סטודיו' , '2 חדרים', '2.5 חדרים' , 'בוגרשוב', 'כיכר', 'לב העיר', 'רבין', 'פרישמן', 'גורדון']
ignore = ['מתיווך' ,'סאבלט', 'קרקע','שותפים','מכירה', 'למכירה','להחלפה']

groupNames = [ '912651298791978','2092819334342645','1196843027043598', '733810383372996', '485208978351445'
              ,'RentinTLV', '711814002180965', '184920528370332', '1643649089181027' , '295395253832427' , '1786825641345386'
             ,'174312609376409' , '1152139018170361' , 'tel.aviv.dirot' , '1463654897222162' , '584681171701217' 
              ,'571535149671300']

prevFile = pd.read_csv("./results.csv") 
prevURLS = prevFile['link'].unique()

for i in groupNames:
    for post in get_posts(group=i, pages=10, options={"posts_per_page": 50, "allow_extra_requests": False}):
        if post['post_text'] == None:
            continue
        if any(words in post['post_text'] for words in interests):
            if any(words in post['post_text'] for words in ignore):
                continue
            postURL = post['post_url'].replace("https://m.", "https://", 1)
            if postURL in prevURLS:
                print("skipped URL "+ postURL)
           #     continue
            text.append(post['post_text'][:200])
            url.append(postURL)
            date.append(post['time'])
            comments.append(post['comments'])
            likes.append(post['likes'])
            user.append(post['username'])

print("Finished parsing facebook results")

if len(date) == 0:
    print("no data for file")
    exit()
    
data = {
    'date': date,
    'link': url,
    'info': text,
    'username': user,
    '#comments': comments,
    '#likes': likes
}

df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
today = pd.to_datetime(datetime.now())
threeDaysAgo = pd.to_datetime(datetime.today() - timedelta(hours=5))
df = df.loc[(df['date'] > threeDaysAgo) & (df['date'] < datetime.now())]
df = df.sort_values('date', ascending=False)
df.dropna
df = df.drop_duplicates(subset='info')
df.to_csv('./results.csv', index=False, encoding='utf-8-sig')

In [ ]:
def sendWhatsapp():
    print("sending whatsapp")
    try:
        send_time = datetime.now()
        send_time_hour = int(send_time.strftime("%H"))
        send_time_min = int(send_time.strftime("%M"))
        NUMBER = "<INSERT YOUR PHONE NUMBER AS STRING INCLUDING COUNTRY CODE (+XYZ...)>"
        urls = df['link'].unique()
        msg = "Appartments:\n\n" + "\n".join(f"{index+1}. {url}" for index, url in enumerate(urls))

        # Print URLs with line numbers
        print("URLs:")
        for index, url in enumerate(urls):
            print(f"{index+1}. {url}")

        # sending message to receiver
        pywhatkit.sendwhatmsg_instantly(NUMBER, msg, tab_close=True)

        print("Successfully Sent!")
    except:
        # handling exception
        # and printing error message
        print("An Unexpected Error!")

In [ ]:
sendWhatsapp()

